In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import shutil
import random
import torch
from PIL import Image

In [ ]:
# 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./models/main_model.pt', force_reload=True)

## 모델로 바운딩 박스 그리기

In [24]:
image_list = os.listdir('./0_scrapping_img')

for f in image_list:
    
    img = Image.open(f'./0_scrapping_img/{f}')
    
    results = model(img)
    
    results.save()
    
    shutil.move(f'./runs/detect/exp/{f}', './1_bounding_img')
    os.rmdir('./runs/detect/exp')

Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 im

## 수작업으로 모델 조차 이상하게 그린 바운딩 박스 이미지 제거

### 0과 1을 대조하여 둘 다 존재하는 이미지의 원본 2로 복사

In [25]:
bounding_processing_img = os.listdir('./1_bounding_img')

scrapping_img = os.listdir('./0_scrapping_img')

for spi in scrapping_img:
    if spi in bounding_processing_img:
        shutil.copy(f'./0_scrapping_img/{spi}', './2_learning_img')

## 세트 분할

In [26]:
image_list = os.listdir('./2_learning_img')

test_set = {}

for i in random.sample(image_list, len(image_list)//10 *2):
    test_set[i] = True

for f_name in image_list:
    path = ''
    
    if test_set.get(f_name):
        path = './3_one_time_data/test/images/'
    else:
        path = './3_one_time_data/train/images/'
    
    shutil.move(f'./2_learning_img/{f_name}', path+f_name)

## 데이터 증강

### 증강 함수 정의

In [27]:
def adjust_hue(image):
    hue_shift = random.randint(-90, 90)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_image[:,:,0] = (hsv_image[:,:,0] + hue_shift) % 180
    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

# image = cv2.imread('image.jpg')
# hue_adjusted_image = adjust_hue(image)
# cv2.imwrite('hue_adjusted_image.jpg', hue_adjusted_image)


def add_gaussian_noise(image, sigma=30):
    row, col, ch = image.shape
    mean = 0
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = image + gauss
    noisy = np.clip(noisy, 0, 255)
    return noisy.astype(np.uint8)

# image = cv2.imread('path/to/your/image.jpg')
# noisy_image = add_gaussian_noise(image, sigma=50)  # 노이즈 강도를 높이려면 sigma 값을 높여주세요.
# cv2.imwrite('path/to/your/noisy_image.jpg', noisy_image)

In [28]:
for CLS in ['train', 'test']:
    image_list = os.listdir(f'./3_one_time_data/{CLS}/images')
    
    for f_name in image_list:
        
        img = Image.open(f'./3_one_time_data/{CLS}/images/{f_name}')
        cv_img = cv2.imread(f'./3_one_time_data/{CLS}/images/{f_name}')
        
        results = model(img)
        
        if len(results.xywh[0]):
            
            for result in results.xywhn[0]:
                x,y,w,h,score,cls = map(float, result)
            
                with open(f'./3_one_time_data/{CLS}/labels/{f_name[:-4]}.txt', 'a') as txt:
                    txt.write(f'{cls} {round(x, 4)} {round(y, 4)} {round(w, 4)} {round(h, 4)}\n')
        
        else:
            continue

        
        adjust_col_img = adjust_hue(cv_img)
        cv2.imwrite(f'./3_one_time_data/{CLS}/images/{f_name[:-4]}_c.jpg', adjust_col_img)
        
        adjust_noise_image = add_gaussian_noise(cv_img)
        cv2.imwrite(f'./3_one_time_data/{CLS}/images/{f_name[:-4]}_n.jpg', adjust_noise_image)

        shutil.copy(f'./3_one_time_data/{CLS}/labels/{f_name[:-4]}.txt', f'./3_one_time_data/{CLS}/labels/{f_name[:-4]}_c.txt')
        shutil.copy(f'./3_one_time_data/{CLS}/labels/{f_name[:-4]}.txt', f'./3_one_time_data/{CLS}/labels/{f_name[:-4]}_n.txt')

## 이상 없으면 3 -> 5 보내버리기!

In [30]:
for CLS in ['train', 'test']:
    
    for i in ['images', 'labels']:
        
        for f in os.listdir(f'./3_one_time_data/{CLS}/{i}'):
            
            shutil.move(f'./3_one_time_data/{CLS}/{i}/{f}', f'./5_total_data/{CLS}/{i}/{f}')

In [2]:
!python ./yolov5/train.py --img 640 --epochs 1 --data ./yolov5/total_data.yaml --cfg ./yolov5/total_yolov5m.yaml --weight ./yolov5/yolov5m.pt --name red_yello_rose

^C


## bounding_img에 없는 labels.txt 삭제 후, images에 사진 복사

In [ ]:
bounding_img_list = os.listdir('./bounding_img/')
label_list = os.listdir('../data_set/total_set/labels/')

for label in label_list:
    if label[:-3]+'jpg' not in bounding_img_list:
        os.remove(f'../data_set/total_set/labels/{label}')
    else:
        shutil.copy(f'./red_rose/{label[:-4]}.jpg', '../data_set/total_set/images/')

## total_set에서 train_set, test_set으로 분화시키기

In [ ]:
total_set_img_list = os.listdir('../data_set/total_set/images/')

test_cnt = (set_cnt := len(total_set_img_list))//100 * 15

test_idx = random.sample(range(set_cnt), test_cnt)

for t in test_idx:
    img_f = total_set_img_list[t]
    lab_f = total_set_label_list[t]
    
    shutil.move(f'../data_set/total_set/images/{img_f}', '../data_set/test_set/images/')
    shutil.move(f'../data_set/total_set/labels/{lab_f}', '../data_set/test_set/labels/')

total_set_img_list = os.listdir('../data_set/total_set/images/')
total_set_label_list = os.listdir('../data_set/total_set/labels/')

for i in range(len(total_set_label_list)):
    img_f = total_set_img_list[i]
    lab_f = total_set_label_list[i]
    
    shutil.move(f'../data_set/total_set/images/{img_f}', '../data_set/train_set/images/')
    shutil.move(f'../data_set/total_set/labels/{lab_f}', '../data_set/train_set/labels/')